In [1]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [5, 10,15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [2]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils

SOURCE_PATH, DEST_PATH = utils.main_processor(
    data_dir='..\data',  # search project ..\data
    downloads_dir='',  # None searchs Downloads dir, '' omits search
    downloads_limit=0,  # search the first 10 files
    clean_name_override='df_perf_ratios.pkl',  # override filename
    start_file_pattern='df_OHLCV_', # search for files starting with 'df_'
)



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\python310.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5\\lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.10.5', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv', '', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\ping\\Files_win10\\python\\py310\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py310\\stocks\\src']


**Available 'df_OHLCV_' files:**

- (1) `[DATA]` `df_OHLCV_2025-03-10_clean.pkl` <span style='color:#00ffff'>(45.55 MB, 2025-03-10 20:09)</span>

- (2) `[DATA]` `df_OHLCV_2025-03-10_clean_clean.pkl` <span style='color:#00ffff'>(45.55 MB, 2025-03-15 20:13)</span>

- (3) `[DATA]` `df_OHLCV_2025-03-14_clean.pkl` <span style='color:#00ffff'>(25.33 MB, 2025-03-15 20:14)</span>


Input a number to select file (1-3)



    **Selected paths:**
    - Source: `..\data\df_OHLCV_2025-03-14_clean.pkl`  
    - Destination: `..\data\df_perf_ratios.pkl`
    

In [3]:
import pandas as pd

# Load the DataFrame from the pickle file
df = pd.read_pickle(SOURCE_PATH)

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
UBS    2025-03-14      32.73
       2025-03-13      31.71
       2025-03-12      31.94
       2025-03-11      31.38
       2025-03-10      31.88
...                      ...
PCVX   2024-03-21      67.64
       2024-03-20      69.13
       2024-03-19      67.60
       2024-03-18      68.04
       2024-03-15      69.70

[344750 rows x 1 columns]

In [4]:
import utils

list_dfs = utils.get_latest_dfs(df, DAYS_RATIO)

In [5]:
all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")

c:\Users\ping\Files_win10\python\py310\.venv\lib\site-packages\empyrical\stats.py:799: RuntimeWarning: divide by zero encountered in divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)



Combined performance metrics DataFrame:
      Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  \
A     -0.967958   -1.857841  0.850624   -3.070634    -4.131212   0.602240   
AA     9.832251   22.572613  3.843882    3.881416     6.204737   1.847612   
AAPL -10.609635  -10.355952  0.222952   -8.425683    -8.241613   0.238875   
ABBV -14.058346  -11.366243  0.036393    0.082421     0.130006   1.013076   
ABEV   6.300423   13.580392  2.710969    9.692998    29.534024   6.533087   
...         ...         ...       ...         ...          ...        ...   
ZG    -2.300781   -3.165323  0.702736   -2.310841    -2.898739   0.717765   
ZM     1.832998    3.315179  1.356618   -0.148098    -0.199910   0.975417   
ZS     1.851402    3.250818  1.349685    2.400416     3.390267   1.435912   
ZTO   -0.275099   -0.456699  0.958836    0.599488     1.042027   1.094113   
ZTS  -11.861555  -10.493258  0.104667   -5.369924    -5.827059   0.388268   

      Sharpe 15d  Sortino 15d  Ome

In [6]:
# Save the combined DataFrame to pickle file
combined_df.to_pickle(DEST_PATH)
print(f"DataFrame successfully saved to {DEST_PATH}")

DataFrame successfully saved to ..\data\df_perf_ratios.pkl


In [7]:
_symbols = ['AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']

from tabulate import tabulate

# Create a DataFrame with only the selected symbols
selected_df = combined_df.loc[_symbols]

# Display the formatted table
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+=============+===============+================+==============+===============+================+==============+
| AAPL |    -10.609